# Programmatically Access TCGA Data using the Seven Bridges Cancer Genomics Cloud via the Datasets API

TCGA is one of the world’s largest cancer genomics data collections, including more than eleven thousand patients, representing 33 cancers, and over half a million total files. Seven Bridges has created a unified metadata ontology from the diverse cancer studies, made this data available, and provided compute infrastructure to facilitate customized analyses on the Cancer Genomics Cloud (the CGC). The CGC provides powerful methods to query and reproducibly analyze TCGA data - alone or in conjunction with your own data.

We continue to develop new methods of interacting with data on the CGC, however, we also appreciate that sometimes it is useful to be able to analyze data locally, or in an AWS environment that you have configured yourself. While the CGC has undergone thorough testing and is certified as a FISMA-moderate system, if you wish to analyze data in alternative locations, you must take the appropriate steps to ensure your computing environment is secure and compliant with current best practices. If you plan to download large numbers of files for local analysis, we recommend using the download utilities available from the Genomic Data Commons which have been specifically optimized for this purpose.

Below, we provide a tutorial showing how to find and access TCGA data using the Datasets API. Alternatively, you can try to query TCGA data [using a SPARQL query](access_TCGA_on_AWS.ipynb).

## Objective
Let's conduct an investigation to look for female Breast Cancer patients who are alive. We want their associated files to be open data, provide gene expression, and be produced by an experimental strategy of RNA-seq.

## Overview
In this tutorial, we will demonstrate how you can use the [Datasets API](http://docs.cancergenomicscloud.org/docs/datasets-api-overview) to find all open access gene expression files obtained from RNA-Seq analysis of living female Breast Cancer patients.

This method allows you to query the following TCGA entities:
* cases
* analytes
* radiation therapies
* drug therapies
* follow ups
* portions
* aliquots
* samples
* slides
* new tumor events
* files

The Datasets API is well-suited for browsing TCGA data.

## Prerequisites
Before you begin this tutorial, you should:

 1. **Set up your CGC account.** If you haven't already done so, navigate to https://cgc.sbgenomics.com/ and follow these [directions](http://docs.cancergenomicscloud.org/docs/sign-up-for-the-cgc) to register for the CGC. This tutorial uses Open Data, which is available to all CGC users. The same approach can be used by approved researchers to access Controlled Data. Learn more about TCGA data access here.
 2. **Install the Seven Bridges' API Python library.** This tutorial uses the library sevenbridges-python. Learn how to [install it](http://docs.cancergenomicscloud.org/blog/programmatically-access-tcga-data-using-the-seven-bridges-cancer-genomics-cloud) before continuing.
 3. **Obtain your authentication token.** You'll use your authentication token to encode your user credentials when interacting with the CGC programmatically. Learn how to [access your authentication token](http://docs.cancergenomicscloud.org/docs/get-your-authentication-token). It is important to store your authentication token in a safe place as it can be used to access your account. The time and location your token was last used is shown on the developer dashboard. If for any reason you believe your token has been compromised, you can regenerate it at any time.

## Query using the Datasets API
The Datasets API is an API designed around the TCGA data structure and focused on search functionality. You can use the Datasets API to browse TCGA using API requests written in JSON. Queries made using the Datasets API return entities and are particularly suitable for browsing TCGA data.

We'll write a Python script to issue our query into TCGA using the Datasets API. Since the Datasets API is not included in our Python library, sevenbridges-python, we will use two Python modules, json and requests, to interact with it instead. We'll use these modules to write a wrapper around the API request.

In [ ]:
import json
from requests import request

Below, we define a simple function to send and receive JSONs from the API using the correctly formatted HTTP calls. The necessary imports are handled above.

In [ ]:
def api_call(path, method='GET', query=None, data=None, token=None):
     
    base_url = 'https://cgc-datasets-api.sbgenomics.com/datasets/tcga/v0/'
     
    data = json.dumps(data) if isinstance(data, dict) \
    or isinstance(data,list) else None
               
    headers = {
        'X-SBG-Auth-Token': token,
        'Accept': 'application/json',
        'Content-type': 'application/json',
    }
     
    response = request(method, base_url + path, params=query, \
                       data=data, headers=headers)
    response_dict = response.json() if \
    response.json() else {}
 
    if response.status_code / 100 != 2:
        print(response_dict)
        raise Exception('Server responded with status code %s.' % response.status_code)
    return response_dict

Then, provide your authentication token, as shown below. Examples of proper coding of your auth\_token are available for [sevenbridges-python bindings](https://github.com/sbg/okAPI/blob/master/Recipes/CGC/Setup_API_environment.ipynb)

In [ ]:
auth_token = 'Put your authentication token here'

Now, we can define a query in JSON for TCGA data based on its [metadata](http://docs.cancergenomicscloud.org/docs/tcga-metadata).

As specified above, we want to find **female**, **Breast Cancer** patients (**cases**) with a vital status (**alive**) and the associated **files** which are **open-access**, provide **Gene expression**, and came from the **experimental strategy** of **RNA-seq**. We will assign an exact value to the above properties.

#### PROTIP:
Extensive details of the Datasets API calls are available [here](http://docs.cancergenomicscloud.org/docs/query-via-the-datasets-api).

In [ ]:
query_body = {
    "entity": "files",
    "hasAccessLevel" : "Open",
    "hasDataType" : "Gene expression",
    "hasExperimentalStrategy": "RNA-Seq",
    "hasCase": {
        "hasDiseaseType" : "Breast Invasive Carcinoma",
        "hasGender" : "FEMALE",
        "hasVitalStatus" : "Alive"
    }
}

The call below returns a dictionary containing the total number of records.

In [ ]:
total = api_call(method='POST', path ='query/total', \
                 token=auth_token, data=query_body)
print("There are {} files matching the query".format(total['total']))

Now, we can create an initial list of all records, 100 at a time. In the example below, this list is named `files_in_query`. We'll use this initial list to catalogue the data returned by the query.

#### PROTIP
Working with pagination in the API is beautifully described <a href="http://docs.cancergenomicscloud.org/docs/the-cgc-api#section-response-pagination">here</a>.

In [ ]:
files_in_query = []

from __future__ import division
from math import ceil

loops = int(ceil(total['total']/100))

for ii in range(0,loops):
    files_in_query.append(api_call(method='POST', \
                                   path =("query?offset=%i" % (100*ii)), \
                                   token=auth_token, data=query_body))
    print("%3.1f percent of files added" % (100*(ii+1)/loops))
    
# NOTE: each item in file_list is a list of 100 files from the query. Example below:
print('\n \n')
print(files_in_query[0]['_embedded']['files'][0])
print(files_in_query[1]['_embedded']['files'][0])

We've now successfully compiled a list of a catalogue of file ids! We'll access the data in the next step using the CGC API.

## Access TCGA data using the CGC API
In this section, we will use the CGC API to access TCGA data. Since we are using the CGC API (as opposed to the Datasets API in the previous step), we will use the [sevenbridges-python bindings](http://sevenbridges-python.readthedocs.io/en/latest/). You should have already installed this library as described under the Prerequisites section. You may also wish to take a look at the library quickstart guide (http://sevenbridges-python.readthedocs.io/en/latest/quickstart/#authentication-and-configuration) before moving forward. Before initializing the library, we recommend creating a config file to store your authentication token for use by the CGC API.

### Set up your authentication token
Since we're now using the CGC API, we need to provide our authentication credentials. Once you have installed sevenbridges-python, you can authenticate by storing your credentials in a config file, `.sbgrc`. Enter your credentials in the config file, as shown below, replacing the last line with your authentication token:

```bash
[cgc]
api-url = https://cgc-api.sbgenomics.com/v2
auth-token = insert auth token here
```

### Initialize the sevenbridges-python library
We've now installed sevenbridges-python and stored our credentials in a config file. Let's import the `api` class from the official `sevenbridges-python` bindings.

In [ ]:
import sevenbridges as sbg

Let's initialize the `api` object so the API knows our credentials.

In [ ]:
# [USER INPUT] specify platform {cgc, sbg}
prof = 'cgc'


config_file = sbg.Config(profile=prof)
api = sbg.Api(config=config_file)

### Access TCGA data
Now, we loop through the first ten files in the first item of the `files_in_query` list from above using the `id` key.

We will now do the following with these ids:

1. Create a list of files on the CGC. From this point, it would be possible to take action on the CGC. For instance, you can use a bioinformatics workflow or tool on these files and start an analysis.
2. (optional) Generate a list of access links.
3. Access each of the ten files in this list. They will be saved to the Downloads folder in your local directory.

In [ ]:
# 1) Generate a list a file objects from the file_ids list
file_list = []
for f in files_in_query[0]['_embedded']['files'][0:10]:
    file_list.append(api.files.get(id = f['id']))
    print(file_list[-1].name)    
    
# (BRANCH-POINT) Do something AWESOME with these files on the CGC


# 2) (optional) Generate a list of download links
dl_list = []
for f in file_list:
    dl_list.append(f.download_info())

    
# 3) Download each of the files in the list to a _downloads_ folder in your local directory.
import os

dl_dir = 'downloads'
try:
    os.stat(dl_dir)
except:
    os.mkdir(dl_dir)

for f in file_list:
    f.download(path = ("%s/%s" % (dl_dir, f.name)))

## Conclusion
Congratulations! You've learned to query TCGA data hosted on AWS using the Datasets API and then access this data using the CGC API. You can learn more about [querying TCGA with the Datasets API](http://docs.cancergenomicscloud.org/v1.0/docs/query-via-the-datasets-api) on our Knowledge Center.

The next move is yours: take the data to the analytical tool and environment of your choice.